# Serve Model

This notebook defines a prototype Flask web server with a REST API endpoint for scoring intances of data. The model-soring API endpoint uses a trained model that is downloaded from cloud storage (AWS S3) and loaded into memory.

# Imports and Configuration

In [1]:
import re
from datetime import date, datetime
from io import BytesIO
from typing import Tuple

import boto3 as aws
import numpy as np
from botocore.exceptions import ClientError
from flask import Flask, jsonify, make_response, request, Response
from joblib import load
from sklearn.base import BaseEstimator

## Load Latest Model

Load the latest trained model from the project's AWS S3 bucket. We start by defining an efficient helper function.

In [2]:
def download_latest_model(aws_bucket: str) -> Tuple[BaseEstimator, date]:
    """Get latest model from AWS S3 bucket."""
    def _date_from_object_key(key: str) -> date:
        """Extract date from S3 file object key."""
        date_string = re.findall('20[2-9][0-9]-[0-1][0-9]-[0-3][0-9]', key)[0]
        file_date = datetime.strptime(date_string, '%Y-%m-%d').date()
        return file_date

    print(f'downloading latest model data from s3://{aws_bucket}/models')
    try:
        s3_client = aws.client('s3')
        s3_objects = s3_client.list_objects(Bucket=aws_bucket, Prefix='models/')
        object_keys_and_dates = [
            (obj['Key'], _date_from_object_key(obj['Key']))
            for obj in s3_objects['Contents']
        ]
        latest_model_obj = sorted(object_keys_and_dates, key=lambda e: e[1])[-1]
        latest_model_obj_key = latest_model_obj[0]
        object_data = s3_client.get_object(Bucket=aws_bucket, Key=latest_model_obj_key)
        model = load(BytesIO(object_data['Body'].read()))
        dataset_date = latest_model_obj[1]
    except ClientError:
        print(f'failed to download model from s3://{aws_bucket}/models')
    return (model, dataset_date)


Applying `download_latest_model` to the project's S3 bucket.

In [3]:
model, data_date = download_latest_model('bodywork-mlops-project')
print(f'- latest model trained on {data_date}\n')
print(model)

downloading latest model data from s3://bodywork-mlops-project/models
- latest model trained on 2021-01-21

LinearRegression()


## Define REST API Endpoint

Where a single instance of data will be POSTed to the `/score/v1` endpoint as JSON - e.g. `{"X": 50}`.

In [4]:
app = Flask(__name__)

@app.route('/score/v1', methods=['POST'])
def score_data_instance() -> Response:
    """Score incoming data instance using loaded model."""
    features = request.json['X']
    X = np.array(features, ndmin=2)
    prediction = model.predict(X)
    response_data = jsonify({'prediction': prediction[0], 'model_info': str(model)})
    return make_response(response_data)

## Start Test Server

Running the cell below will start the Flask test server and print the logs as results. To test the API endpoint defined above, open a terminal in another window and execute the following HTTP request using the curl tool,

```shell
curl http://0.0.0.0:5000/score/v1 \
    --request POST \
    --header "Content-Type: application/json" \
    --data '{"X": 50}'
```

The successful response from the model-scoring service should look like,

```json
{
    "prediction": 54.57560049377929,
    "model_info": "LinearRegression()"
}
```

Stop the service by interrupting the kernel (i.e. hit the 'stop button' above).

In [ ]:
app.run(host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
